In [ ]:
import pandas as pd

def read_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print("An error occurred:", e)
        return None
    

In [ ]:
data= read_csv_to_dataframe("/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/102_Data.csv")
path_smote="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote"
path_strat="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/strat"
path_smote_strat="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote_strat"


In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pandas as pd

def tune_gradient_boosting_hyperparameters(X_train, y_train, X_test, y_test):
    param_dist = {
        'n_estimators': range(50, 1001),
        'learning_rate': (0.001, 0.3),
        'max_depth': range(3, 12),
        'min_samples_split': range(2, 22),
        'min_samples_leaf': range(1, 22),
        'subsample': (0.8, 1.0),
        'max_features': ['sqrt', 'log2']
    }

    gb_classifier = GradientBoostingClassifier(random_state=42)

    scaler = StandardScaler()


    X_train_scaled = scaler.fit_transform(X_train)

    X_test_scaled = scaler.transform(X_test)

    cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    randomized_search = RandomizedSearchCV(
        gb_classifier, param_distributions=param_dist, n_iter=50, cv=cv_strategy, scoring='accuracy', random_state=42
    )
    
    randomized_search.fit(X_train, y_train)

    results_df = pd.DataFrame(randomized_search.cv_results_)

    results_df.to_csv('hyperparameter_tuning_results_randomized_gb.csv', index=False)

    print(f"Best Hyperparameters: {randomized_search.best_params_}")
    print(f"Best Accuracy: {randomized_search.best_score_}")

    y_train_pred = randomized_search.best_estimator_.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    print(f"Train Accuracy: {train_accuracy}")

    y_test_pred = randomized_search.best_estimator_.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy}")
    return randomized_search.best_params_

# Assuming X_train, y_train, X_test, and y_test are available
params = tune_gradient_boosting_hyperparameters(X_train, y_train, X_test, y_test)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.24006647904054476, max_depth=7, max_features='sqrt', min_samples_leaf=15, min_samples_split=19, n_estimators=731, subsample=0.8848356144921364, random_state=42)

In [ ]:
from Utility_model_training import evaluate_classifier_with_stratified_kfold, evaluate_classifier_with_kfold_smote, evaluate_classifier_with_stratified_smote
evaluate_classifier_with_stratified_smote(X_train, y_train, X_test, y_test, gb, num_folds=10,save_path=path_smote_strat,model_name="gb_classifier_smote_stratified")
evaluate_classifier_with_stratified_kfold(X_train, y_train, X_test, y_test, gb, num_folds=10,save_path=path_strat,model_name="gb_classifier_stratified")
evaluate_classifier_with_kfold_smote(X_train, y_train, X_test, y_test, gb, num_folds=10,save_path=path_smote,model_name="gb_classifier_smote")

In [ ]:
path_smote_strat_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote_strat/ROC_Curves"
path_strat_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/strat/ROC_Curves"
path_smote_ROC = "/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/models_102/smote/ROC_Curves"
from Utility_ROC_Curves import plot_roc_curves_multiclass_smote_strat,plot_roc_curves_multiclass_smote, plot_roc_curves_multiclass_strat
plot_roc_curves_multiclass_smote_strat(gb, X, y, n_splits=10, save_folder=path_smote_strat_ROC, model_name="gb_KFold_SMOTE_strat")
plot_roc_curves_multiclass_strat(gb, X, y, n_splits=10, save_folder=path_strat_ROC, model_name="gb_KFold_Strat")
plot_roc_curves_multiclass_smote(gb, X, y, n_splits=10, save_folder=path_smote_ROC, model_name="gb_KFold_SMOTE")
